# Tutorial: Model-Based Evaluation of RAG Pipelines

- **Level**: Beginner
- **Time to complete**: 10 minutes
- **Components Used**: `InMemoryDocumentStore`, `InMemoryBM25Retriever`, `PromptBuilder`, `OpenAIGenerator`, `UpTrainEvaluator`
- **Prerequisites**: You must have an API key from an active OpenAI account as this tutorial is using the gpt-3.5-turbo model by OpenAI.
- **Goal**: After completing this tutorial, you'll have learned how to evaluate your RAG pipelines using some of the model-based evaluation frameworkes integerated into Haystack.

> This tutorial uses Haystack 2.0 Beta. To learn more, read the [ Haystack 2.0 Beta announcement](https://haystack.deepset.ai/blog/introducing-haystack-2-beta-and-advent) or see [Haystack 2.0 Documentation](https://docs.haystack.deepset.ai/v2.0/docs).

## Overview

This tutorial shows you how to evaluate a generative question-answering pipeline that uses the retrieval-augmentation ([RAG](https://www.deepset.ai/blog/llms-retrieval-augmentation)) approach with Haystack 2.0. As we're doing model-based evaluation, no ground-truth labels are required. The process involves Haystack's integration of three evaluation frameworks:
- [UpTrain](https://docs.uptrain.ai/getting-started/introduction) ✅
- [RAGAS](https://docs.ragas.io/en/stable/index.html) 🔜
- [DeepEval](https://docs.confident-ai.com/docs/getting-started) 🔜

For this tutorial, you'll use the Wikipedia pages of [Seven Wonders of the Ancient World](https://en.wikipedia.org/wiki/Wonders_of_the_World) as Documents, but you can replace them with any text you want.


## Preparing the Colab Environment

- [Enable GPU Runtime in Colab](https://docs.haystack.deepset.ai/docs/enabling-gpu-acceleration#enabling-the-gpu-in-colab)
- [Set logging level to INFO](https://docs.haystack.deepset.ai/docs/log-level)

## Installing Haystack

Install Haystack 2.0 Beta and [datasets](https://pypi.org/project/datasets/) with `pip`:

In [ ]:
%%bash

pip install haystack-ai
pip install "datasets>=2.6.1"
pip install uptrain-haystack

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.1/238.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 7.8 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


### Enabling Telemetry

Knowing you're using this tutorial helps us decide where to invest our efforts to build a better product but you can always opt out by commenting the following line. See [Telemetry](https://docs.haystack.deepset.ai/v2.0/docs/enabling-telemetry) for more details.

In [ ]:
from haystack.telemetry import tutorial_running

tutorial_running(34)

## Create the RAG Pipeline to Evaluate

To evaluate a RAG pipeline, we need a RAG pipeline to start with. So, we will start by creating a question answering pipelne.

> 💡 For a complete tutorial on creating Retrieval-Augmmented Generation pipelines check out the [Creating Your First QA Pipeline with Retrieval-Augmentation Tutorial](https://haystack.deepset.ai/tutorials/27_first_rag_pipeline)

First, we will initialize a DocumentStore. A DocumentStore stores the Documents that the question answering system uses to find answers to your questions. In this tutorial, you'll be using the `InMemoryDocumentStore`.

You'll use the Wikipedia pages of [Seven Wonders of the Ancient World](https://en.wikipedia.org/wiki/Wonders_of_the_World) as Documents. We preprocessed the data and uploaded to a Hugging Face Space: [Seven Wonders](https://huggingface.co/datasets/bilgeyucel/seven-wonders). Thus, you don't need to perform any additional cleaning or splitting.


In [ ]:
from datasets import load_dataset
from haystack import Document
from haystack.document_stores.in_memory import InMemoryDocumentStore

document_store = InMemoryDocumentStore()


dataset = load_dataset("bilgeyucel/seven-wonders", split="train")
docs = [Document(content=doc["content"], meta=doc["meta"]) for doc in dataset]
document_store.write_documents(docs)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

151

> `InMemoryDocumentStore` is the simplest DocumentStore to get started with. It requires no external dependencies and it's a good option for smaller projects and debugging. But it doesn't scale up so well to larger Document collections, so it's not a good choice for production systems. To learn more about the different types of external databases that Haystack supports, see [DocumentStore Integrations](https://haystack.deepset.ai/integrations?type=Document+Store).

Now that we have our data ready, we can create a simple RAG pipeline.

In this example, we'll be using:
- [`InMemoryBM25Retriever`](https://docs.haystack.deepset.ai/v2.0/docs/inmemorybm25retriever) which will get the relevant documents to the query.
- [`OpenAIGenerator`](https://docs.haystack.deepset.ai/v2.0/docs/OpenAIGenerator) to generate answers to queries. You can replace `OpenAIGenerator` in your pipeline with another `Generator`. Check out the full list of generators [here](https://docs.haystack.deepset.ai/v2.0/docs/generators).

In [ ]:
import os
from getpass import getpass
from haystack.components.builders import PromptBuilder
from haystack.components.generators import OpenAIGenerator
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever

retriever = InMemoryBM25Retriever(document_store)

template = """
Given the following information, answer the question.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{question}}
Answer:
"""

prompt_builder = PromptBuilder(template=template)


os.environ['OPENAI_API_KEY']  = getpass("Enter OpenAI API key:")
generator = OpenAIGenerator()

Enter OpenAI API key:··········



To build a pipeline, add all components to your pipeline and connect them. Create connections from `retriever` to the `prompt_builder` and from `prompt_builder` to `llm`. Explicitly connect the output of `retriever` with "documents" input of the `prompt_builder` to make the connection obvious as `prompt_builder` has two inputs ("documents" and "question"). For more information on pipelines and creating connections, refer to [Creating Pipelines](https://docs.haystack.deepset.ai/v2.0/docs/creating-pipelines) documentation.

In [ ]:
from haystack.pipeline import Pipeline
from haystack.components.builders.answer_builder import AnswerBuilder

rag_pipeline = Pipeline()
# Add components to your pipeline
rag_pipeline.add_component("retriever", retriever)
rag_pipeline.add_component("prompt_builder", prompt_builder)
rag_pipeline.add_component("llm", generator)
rag_pipeline.add_component(instance=AnswerBuilder(), name="answer_builder")

# Now, connect the components to each other
rag_pipeline.connect("retriever", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder", "llm")
rag_pipeline.connect("llm.replies", "answer_builder.replies")
rag_pipeline.connect("llm.meta", "answer_builder.meta")
rag_pipeline.connect("retriever", "answer_builder.documents")

That's it! The pipeline's ready to generate answers to questions!

### Asking a Question

When asking a question, use the `run()` method of the pipeline. Make sure to provide the question to both the `retriever` and the `prompt_builder`. This ensures that the `{{question}}` variable in the template prompt gets replaced with your specific question.

In [ ]:
question = "When was the Rhodes Statue built?"

response = rag_pipeline.run({"retriever": {"query": question}, "prompt_builder": {"question": question}, "answer_builder": {"query": question}})

Ranking by BM25...:   0%|          | 0/151 [00:00<?, ? docs/s]

In [ ]:
print(response['answer_builder']['answers'][0].data)

The Rhodes Statue was built in 280 BC.


Now that we have the RAG pipeline ready, we create an evaluation pipeline. It defines what metrics we want to evaluate and we choose one of Haystack's integrated evalaution frameworks.

## Evaluate The Pipeline with UpTrain

Now that we have a RAG pipeline, let's look at how we can evaluate it. Here, we're using the Haystack UpTrain integration. We will perform 2 evaluations:

- Context Relevance
- Critique language

For a full list of available metrics and their expected inputs, check out our [`UpTrainEvaluator` Docs]().

#### 1) Evaluate Context Relevance

In [ ]:
from haystack_integrations.components.evaluators import UpTrainEvaluator, UpTrainMetric

evaluator = UpTrainEvaluator(
    metric=UpTrainMetric.CONTEXT_RELEVANCE,
    api="openai",
)

evaluator_pipeline = Pipeline()
evaluator_pipeline.add_component("evaluator", evaluator)

Next, we can create a helper function to evaluate the context relevance of a RAG pipeline with multiple questions. The context relevance metric expects 2 inputs that should be provided from the RAG pipeline we are evaluating:

- questions
- contexts

In [ ]:
def evaluate_context_relevance(questions, evaluation_pipeline):
  contexts = []
  responses = []
  for question in questions:
      response = rag_pipeline.run({"retriever": {"query": question},
                                   "prompt_builder": {"question": question},
                                   "answer_builder": {"query": question}})
      context = ""
      for d in response['answer_builder']['answers'][0].documents:
        context+= d.content

      contexts.append([context])
      responses.append(response["answer_builder"]["answers"][0].data)


  evaluation_results = evaluator_pipeline.run({"evaluator": {"questions": questions,
                                                             "contexts": contexts}})
  return evaluation_results

In [ ]:
questions = ["When was the Rhodes Statue built?",
             "Where is the Pyramid of Giza?",
             "When was the pyramid built?"]

evaluate_context_relevance(questions=questions, evaluation_pipeline=evaluator_pipeline)

Ranking by BM25...:   0%|          | 0/151 [00:00<?, ? docs/s]

Ranking by BM25...:   0%|          | 0/151 [00:00<?, ? docs/s]

Ranking by BM25...:   0%|          | 0/151 [00:00<?, ? docs/s]

2024-02-06 13:59:29.446 | INFO     | uptrain.framework.evalllm:evaluate:100 - Sending evaluation request for rows 0 to <50 to the Uptrain
2024-02-06 13:59:50.854 | INFO     | uptrain.framework.evalllm:evaluate:100 - Sending evaluation request for rows 0 to <50 to the Uptrain
2024-02-06 14:00:05.119 | INFO     | uptrain.framework.evalllm:evaluate:100 - Sending evaluation request for rows 0 to <50 to the Uptrain


{'evaluator': {'results': [[{'name': 'context_relevance',
     'score': 1.0,
     'explanation': ' "The extracted context mentions that the construction of the Rhodes Statue began in 292 BC, providing a specific date for the start of the construction. It also states that the statue collapsed in 226 BC, giving a specific date for its destruction. Therefore, the extracted context can answer the given user query completely by providing the specific dates related to the construction and destruction of the Rhodes Statue."\n'}],
   [{'name': 'context_relevance',
     'score': 1.0,
     'explanation': "The extracted context clearly states that the Pyramid of Giza is located in Giza, Egypt, at the most Northern end of the line of the 3 Pyramids of Giza. It also provides additional details about the pyramid's historical significance, construction, and surrounding structures, thereby answering the user query completely."}],
   [{'name': 'context_relevance',
     'score': 0.5,
     'explanation':

#### 2) Critique Tone

An evaluator that uses the `CRITIQUE_TONE` metric expects to be initlaized with an `llm_persona`

In [ ]:
from haystack_integrations.components.evaluators import UpTrainEvaluator, UpTrainMetric

evaluator = UpTrainEvaluator(
    metric=UpTrainMetric.CRITIQUE_TONE,
    api="openai",
    metric_params={"llm_persona": "informative"}
)

evaluator_pipeline = Pipeline()
evaluator_pipeline.add_component("evaluator", evaluator)

Next, we can create a helper function to critiqye the tone of the results of a RAG pipeline. This metric expects 1 input that should be provided from the RAG pipeline we are evaluating:

- responses

In [ ]:
def evaluate_critique_tone(questions, evaluation_pipeline):
  responses = []
  for question in questions:
      response = rag_pipeline.run({"retriever": {"query": question},
                                   "prompt_builder": {"question": question},
                                   "answer_builder": {"query": question}})
      responses.append(response["answer_builder"]["answers"][0].data)


  evaluation_results = evaluator_pipeline.run({"evaluator": {"responses": responses}})
  return evaluation_results

In [ ]:
questions = ["When was the Rhodes Statue built?",
             "Where is the Pyramid of Giza?",
             "When was the pyramid built?"]

evaluate_critique_tone(questions=questions, evaluation_pipeline=evaluator_pipeline)

Ranking by BM25...:   0%|          | 0/151 [00:00<?, ? docs/s]

Ranking by BM25...:   0%|          | 0/151 [00:00<?, ? docs/s]

Ranking by BM25...:   0%|          | 0/151 [00:00<?, ? docs/s]

2024-02-06 14:51:03.048 | INFO     | uptrain.framework.evalllm:evaluate:100 - Sending evaluation request for rows 0 to <50 to the Uptrain
2024-02-06 14:51:07.514 | INFO     | uptrain.framework.evalllm:evaluate:100 - Sending evaluation request for rows 0 to <50 to the Uptrain
2024-02-06 14:51:09.276 | INFO     | uptrain.framework.evalllm:evaluate:100 - Sending evaluation request for rows 0 to <50 to the Uptrain


{'evaluator': {'results': [[{'name': 'tone',
     'score': 0.6,
     'explanation': 'The response is factual and informative, providing a specific detail about the Rhodes Statue. However, it lacks additional context or explanation, which could enhance its informativeness. \n[Score]: 3'}],
   [{'name': 'tone',
     'score': 1.0,
     'explanation': '[Persona]: informative\n[Response]: The response is straightforward and provides factual information without any additional explanation or context. It aligns with the informative persona as it delivers a clear and concise piece of information.\n[Score]: 5'}],
   [{'name': 'tone',
     'score': 1.0,
     'explanation': '[Persona]: informative\n[Response]: The pyramid was built in the early 26th century BC.\n[Reasoning]: The response is straightforward and provides a specific piece of information without any additional context or explanation. It aligns well with the informative persona as it delivers a factual statement. \n[Score]: 5'}]]}}

## What's next

🎉 Congratulations! You've learned how to evaluate a RAG pipeline with model-based evaluation frameworks and without any labeling efforts.

If you liked this tutorial, you may also enjoy:
- [Serializing Haystack Pipelines](https://haystack.deepset.ai/tutorials/29_serializing_pipelines)
-  [Creating Your First QA Pipeline with Retrieval-Augmentation](https://haystack.deepset.ai/tutorials/27_first_rag_pipeline)

To stay up to date on the latest Haystack developments, you can [sign up for our newsletter](https://landing.deepset.ai/haystack-community-updates?utm_campaign=developer-relations&utm_source=moel_based_evaluation). Thanks for reading!